## Load Data

In [ ]:
from src.morse import Morse
from src.evaluation_and_conversion import compare_result_txt_to_groundtruth_ply
import itertools

In [ ]:
def process(artifact, persistence, params, index = 7):
    data_conf = Morse()
    data_n = Morse()
    data_conf.load_mesh_ply("Data/"+artifact+".ply", quality_index=index, inverted=True)
    data_n.load_mesh_ply("Data/"+artifact+".ply", quality_index=index, inverted=True)

    outfilename =  "Data/results/" + artifact + "/" + artifact

    #### CONFORMING
    data_conf.load_labels("Data/"+artifact+"_labels.txt")
    data_conf.ConformingGradient()
    data_conf.ExtractMorseComplex()

    accuracy = 0
    best_params = []

    for pers in persistence:
        data_conf.ReduceMorseComplex(pers, conforming=True)
        data_conf.ExtractMorseCells(pers)

        for high, low, merge in params:
            if high > low:
                data_conf.Segmentation(pers, high, low, merge, conforming=True)
                data_conf.plot_Segmentation_label_txt(pers, high, low, merge, outfilename)
                result = compare_result_txt_to_groundtruth_ply(outfilename+ "_" + str(pers) + "P_" + str(high) + "-" + str(low) 
                                       + "T_" + str(merge) + ".txt", "Data/"+artifact+"_c.ply")
                if result[0] > accuracy:
                    accuracy = result[0]
                    best_params = result[1:]

    print(accuracy, best_params)

    ### NON CONFORMING
    data_n.ProcessLowerStars()
    data_n.ExtractMorseComplex()

    data_n.ReduceMorseComplex(best_params[0], conforming=False)
    data_n.ExtractMorseCells(best_params[0])
    data_n.Segmentation(best_params[0], best_params[1], best_params[2], best_params[3], conforming=False)
    data_n.plot_Segmentation_label_txt(best_params[0], best_params[1], best_params[2], best_params[3], outfilename+"non_conf")




In [ ]:
# PARAMETERS
persistence = [0.03, 0.04, 0.05]
high_thresh = [0.06, 0.07, 0.08]
low_thresh = [0.02, 0.03, 0.04, 0.05]
merge_thresh = [0.3, 0.4, 0.5]


process("700", persistence, list(itertools.product(high_thresh, low_thresh, merge_thresh)))

In [ ]:
# PARAMETERS
persistence = [0.03, 0.04, 0.05]
high_thresh = [0.04, 0.05, 0.06]
low_thresh = [0.02, 0.03, 0.04]
merge_thresh = [0.1, 0.2, 0.3]

process("801", persistence, list(itertools.product(high_thresh, low_thresh, merge_thresh)), index = 3)

In [ ]:
# PARAMETERS
persistence = [0.06, 0.07, 0.08]
high_thresh = [0.08, 0.09, 0.1]
low_thresh = [0.06, 0.07, 0.08]
merge_thresh = [0.2, 0.3, 0.4]

process("4501", persistence, list(itertools.product(high_thresh, low_thresh, merge_thresh)))